In [1]:
import pandas as pd
from datetime import datetime
import os

In [2]:
APPLICATION_BASE_PATH = "I:/PhD/Model_Building/Stage_4_Building_Vulnerability_Dataset"
ROUND = "7_Round"

# APP_FILES="App"
APP_FILES="Files"

round_path = APPLICATION_BASE_PATH+"/"+ROUND+"_Qark"
data_csv = round_path+"/"+ROUND+"_"+APP_FILES+"_Qark_Analysis_CWE_mapped.csv"
datasets_path =round_path+"/Scanned_"+APP_FILES+"_Unprocessed_Dataset_Parts"

TYPE = "Scanned_"+APP_FILES

In [3]:
df_data = pd.read_csv(data_csv)

In [4]:
df_data.head()

,Unnamed: 0,app_name,category,severity,description,qark_name,line_number,file_object,apk_exploit_dict,CWE_ID,CWE_Desc
0,0,us-travel-converter,file,INFO,Application contains hardcoded HTTP url: http:...,Hardcoded HTTP url found,[32_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-312,Cleartext Storage of Sensitive Information
1,1,us-travel-converter,file,INFO,Application contains hardcoded HTTP url: http:...,Hardcoded HTTP url found,[37_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-312,Cleartext Storage of Sensitive Information
2,2,userland,file,INFO,Please confirm and investigate the API key to ...,Potential API Key found,[83_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-200,Exposure of Sensitive Information to an Unauth...
3,3,userland,file,INFO,Please confirm and investigate the API key to ...,Potential API Key found,[211_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-200,Exposure of Sensitive Information to an Unauth...
4,4,userland,file,INFO,Please confirm and investigate the API key to ...,Potential API Key found,[125_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-200,Exposure of Sensitive Information to an Unauth...


In [5]:
df_data.set_index('app_name')
df_data.head()

,Unnamed: 0,app_name,category,severity,description,qark_name,line_number,file_object,apk_exploit_dict,CWE_ID,CWE_Desc
0,0,us-travel-converter,file,INFO,Application contains hardcoded HTTP url: http:...,Hardcoded HTTP url found,[32_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-312,Cleartext Storage of Sensitive Information
1,1,us-travel-converter,file,INFO,Application contains hardcoded HTTP url: http:...,Hardcoded HTTP url found,[37_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-312,Cleartext Storage of Sensitive Information
2,2,userland,file,INFO,Please confirm and investigate the API key to ...,Potential API Key found,[83_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-200,Exposure of Sensitive Information to an Unauth...
3,3,userland,file,INFO,Please confirm and investigate the API key to ...,Potential API Key found,[211_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-200,Exposure of Sensitive Information to an Unauth...
4,4,userland,file,INFO,Please confirm and investigate the API key to ...,Potential API Key found,[125_ 0],I:\PhD\Model_Building\APKs_and_GitRepos\GITRep...,None,CWE-200,Exposure of Sensitive Information to an Unauth...


In [6]:
df_data.shape

(14668, 11)

In [7]:
def read_source_file(file_path):
    file_df = pd.read_csv(file_path, names=['code_line'], sep="\t", skip_blank_lines=False).fillna("")
    return file_df

In [8]:
def generate_dataset(start, end):
    vulberability_dataset = []  
    for i in range(start, end):
        print("Processing line :",i)
        row = df_data.iloc[i]
        try:
            app_name = row.app_name
            category = row.category
            severity = row.severity
            qark_name = row.qark_name    
            line_number = int(str(row.line_number).split("_")[0][1:])
            file_object = row.file_object
            apk_exploit_dict = row.apk_exploit_dict
            CWE_ID = row.CWE_ID
            CWE_Desc = row.CWE_Desc 
            try:

                source_file = read_source_file(file_object)
                file_type = str(file_object).split(".")[1].lower()
                if(file_type not in ['java', 'kt', 'xml', 'txt']):
                    continue

                for j, file_row in source_file.iterrows():
                    code_line = file_row.code_line.strip()
                    if(j == line_number-1):
                        vulnerability_status = 1

                        csv_data_row = [app_name, category, severity, qark_name, code_line, CWE_ID, CWE_Desc,vulnerability_status]
                    else:
                        vulnerability_status = 0
                        csv_data_row = [app_name, "", "", "", code_line, "", "",vulnerability_status]       


                    vulberability_dataset.append(csv_data_row)
            except Exception as e:
                print(e)
                continue
        except Exception as ef:
            print(ef)
            continue
        print("Processed line :",i)


    header = ['app_name', 'category', 'severity', 'qark_name', 'Code', 'CWE_ID', 'CWE_Desc', 'Vulnerability_status']
    vulberability_dataframe = pd.DataFrame(vulberability_dataset, columns = header)

    vulberability_dataframe.drop_duplicates(subset=['Code', 'CWE_ID'], inplace=True)

    vulberability_dataframe.to_csv(datasets_path+"/"+TYPE+'_Unprocessed_Dataset_'+str(start)+'_'+str(end)+'.csv', sep=',', encoding='utf-8', index=False)


In [9]:
def execute_generation_process():
    dataset_length = len(df_data)
    increment = 2500

    for i in range(0,dataset_length, increment):
        start = i
        end = i+increment
        if(end>dataset_length):
            end= dataset_length

        print(str(datetime.now())+": Dataset generating : "+ str(start)+"_"+ str(end))
        generate_dataset(start,end)

In [10]:
execute_generation_process()

2022-06-27 13:30:47.910966: Dataset generating : 0_2500
Processing line : 0
Processed line : 0
Processing line : 1
Processed line : 1
Processing line : 2
Processing line : 3
Processing line : 4
Processing line : 5
Processing line : 6
Processing line : 7
Processing line : 8
Processing line : 9
Processing line : 10
Processing line : 11
Processing line : 12
Processed line : 12
Processing line : 13
Processed line : 13
Processing line : 14
Processed line : 14
Processing line : 15
Processed line : 15
Processing line : 16
Processed line : 16
Processing line : 17
Processed line : 17
Processing line : 18
Processed line : 18
Processing line : 19
Processed line : 19
Processing line : 20
Processed line : 20
Processing line : 21
Processed line : 21
Processing line : 22
Processed line : 22
Processing line : 23
Processed line : 23
Processing line : 24
invalid literal for int() with base 10: 'one'
Processing line : 25
Processed line : 25
Processing line : 26
Processed line : 26
Processing line : 27
Pr

Processed line : 202
Processing line : 203
Processed line : 203
Processing line : 204
Processed line : 204
Processing line : 205
Processed line : 205
Processing line : 206
Processed line : 206
Processing line : 207
Processed line : 207
Processing line : 208
Processed line : 208
Processing line : 209
Processed line : 209
Processing line : 210
Processed line : 210
Processing line : 211
Processed line : 211
Processing line : 212
invalid literal for int() with base 10: 'one'
Processing line : 213
Processed line : 213
Processing line : 214
Processed line : 214
Processing line : 215
Processed line : 215
Processing line : 216
Processed line : 216
Processing line : 217
Processed line : 217
Processing line : 218
Processed line : 218
Processing line : 219
Processed line : 219
Processing line : 220
Processed line : 220
Processing line : 221
Processed line : 221
Processing line : 222
Processed line : 222
Processing line : 223
invalid literal for int() with base 10: 'one'
Processing line : 224
Proc

Processed line : 390
Processing line : 391
Processed line : 391
Processing line : 392
Processed line : 392
Processing line : 393
Processed line : 393
Processing line : 394
Processed line : 394
Processing line : 395
Processed line : 395
Processing line : 396
Processed line : 396
Processing line : 397
Processed line : 397
Processing line : 398
Processed line : 398
Processing line : 399
Processed line : 399
Processing line : 400
Processed line : 400
Processing line : 401
Processed line : 401
Processing line : 402
Processed line : 402
Processing line : 403
Processed line : 403
Processing line : 404
Processed line : 404
Processing line : 405
Processed line : 405
Processing line : 406
Processed line : 406
Processing line : 407
Processed line : 407
Processing line : 408
Processed line : 408
Processing line : 409
Processed line : 409
Processing line : 410
Processed line : 410
Processing line : 411
Processed line : 411
Processing line : 412
Processed line : 412
Processing line : 413
Processed l

Processing line : 629
Processing line : 630
Processing line : 631
Processing line : 632
Processing line : 633
Processing line : 634
Processing line : 635
Processing line : 636
Processed line : 636
Processing line : 637
Processing line : 638
Processing line : 639
Processing line : 640
Processing line : 641
Processed line : 641
Processing line : 642
Processing line : 643
Processing line : 644
Processing line : 645
Processing line : 646
Processing line : 647
Processing line : 648
Processing line : 649
Processing line : 650
Processed line : 650
Processing line : 651
Processed line : 651
Processing line : 652
Processed line : 652
Processing line : 653
Processing line : 654
Processing line : 655
Processing line : 656
Processing line : 657
Processing line : 658
Processing line : 659
invalid literal for int() with base 10: 'one'
Processing line : 660
Processing line : 661
Processed line : 661
Processing line : 662
Processing line : 663
Processing line : 664
Processing line : 665
Processing lin

Processing line : 922
Processing line : 923
Processing line : 924
Processing line : 925
Processing line : 926
Processing line : 927
Processing line : 928
Processing line : 929
Processing line : 930
Processing line : 931
Processing line : 932
Processing line : 933
Processing line : 934
Processing line : 935
Processing line : 936
Processing line : 937
Processing line : 938
Processing line : 939
Processing line : 940
Processing line : 941
Processing line : 942
Processing line : 943
Processing line : 944
Processing line : 945
Processing line : 946
Processing line : 947
Processing line : 948
Processing line : 949
Processing line : 950
Processing line : 951
Processing line : 952
Processing line : 953
Processed line : 953
Processing line : 954
Processing line : 955
Processing line : 956
Processing line : 957
Processing line : 958
Processing line : 959
Processing line : 960
Processing line : 961
Processing line : 962
Processing line : 963
Processing line : 964
Processing line : 965
Processing 

...

Processed line : 13904
Processing line : 13905
Processed line : 13905
Processing line : 13906
Processed line : 13906
Processing line : 13907
Processed line : 13907
Processing line : 13908
Processed line : 13908
Processing line : 13909
Processed line : 13909
Processing line : 13910
Processed line : 13910
Processing line : 13911
Processed line : 13911
Processing line : 13912
Processed line : 13912
Processing line : 13913
Processed line : 13913
Processing line : 13914
Processed line : 13914
Processing line : 13915
Processed line : 13915
Processing line : 13916
Processed line : 13916
Processing line : 13917
Processed line : 13917
Processing line : 13918
Processed line : 13918
Processing line : 13919
Processed line : 13919
Processing line : 13920
Processed line : 13920
Processing line : 13921
Processed line : 13921
Processing line : 13922
Processed line : 13922
Processing line : 13923
Processed line : 13923
Processing line : 13924
Processed line : 13924
Processing line : 13925
Processed lin

Processed line : 14083
Processing line : 14084
Processed line : 14084
Processing line : 14085
Processed line : 14085
Processing line : 14086
Processed line : 14086
Processing line : 14087
Processed line : 14087
Processing line : 14088
Processed line : 14088
Processing line : 14089
Processed line : 14089
Processing line : 14090
invalid literal for int() with base 10: 'one'
Processing line : 14091
Processed line : 14091
Processing line : 14092
Processed line : 14092
Processing line : 14093
Processed line : 14093
Processing line : 14094
Processed line : 14094
Processing line : 14095
invalid literal for int() with base 10: 'one'
Processing line : 14096
Processed line : 14096
Processing line : 14097
Processed line : 14097
Processing line : 14098
Processed line : 14098
Processing line : 14099
Processed line : 14099
Processing line : 14100
Processed line : 14100
Processing line : 14101
Processed line : 14101
Processing line : 14102
Processed line : 14102
Processing line : 14103
Processed line

Processed line : 14249
Processing line : 14250
Processed line : 14250
Processing line : 14251
Processed line : 14251
Processing line : 14252
Processed line : 14252
Processing line : 14253
Processed line : 14253
Processing line : 14254
invalid literal for int() with base 10: 'one'
Processing line : 14255
Processed line : 14255
Processing line : 14256
Processed line : 14256
Processing line : 14257
Processed line : 14257
Processing line : 14258
Processed line : 14258
Processing line : 14259
Processed line : 14259
Processing line : 14260
Processed line : 14260
Processing line : 14261
Processed line : 14261
Processing line : 14262
Processed line : 14262
Processing line : 14263
Processed line : 14263
Processing line : 14264
Processed line : 14264
Processing line : 14265
invalid literal for int() with base 10: 'one'
Processing line : 14266
Processed line : 14266
Processing line : 14267
invalid literal for int() with base 10: 'one'
Processing line : 14268
Processed line : 14268
Processing line

Processed line : 14422
Processing line : 14423
Processed line : 14423
Processing line : 14424
Processed line : 14424
Processing line : 14425
Processed line : 14425
Processing line : 14426
Processed line : 14426
Processing line : 14427
Processed line : 14427
Processing line : 14428
Processed line : 14428
Processing line : 14429
Processed line : 14429
Processing line : 14430
Processed line : 14430
Processing line : 14431
Processed line : 14431
Processing line : 14432
Processed line : 14432
Processing line : 14433
Processed line : 14433
Processing line : 14434
Processed line : 14434
Processing line : 14435
Processed line : 14435
Processing line : 14436
Processed line : 14436
Processing line : 14437
Processed line : 14437
Processing line : 14438
Processed line : 14438
Processing line : 14439
Processed line : 14439
Processing line : 14440
Processed line : 14440
Processing line : 14441
Processed line : 14441
Processing line : 14442
Processed line : 14442
Processing line : 14443
Processed lin

Processed line : 14606
Processing line : 14607
Processed line : 14607
Processing line : 14608
Processed line : 14608
Processing line : 14609
Processed line : 14609
Processing line : 14610
Processed line : 14610
Processing line : 14611
Processed line : 14611
Processing line : 14612
Processed line : 14612
Processing line : 14613
Processed line : 14613
Processing line : 14614
Processed line : 14614
Processing line : 14615
Processed line : 14615
Processing line : 14616
Processed line : 14616
Processing line : 14617
Processed line : 14617
Processing line : 14618
Processed line : 14618
Processing line : 14619
Processed line : 14619
Processing line : 14620
Processed line : 14620
Processing line : 14621
Processed line : 14621
Processing line : 14622
Processed line : 14622
Processing line : 14623
Processed line : 14623
Processing line : 14624
Processed line : 14624
Processing line : 14625
Processed line : 14625
Processing line : 14626
Processed line : 14626
Processing line : 14627
Processed lin

In [11]:
def combine_dfs():
    df_all =  pd.DataFrame()
    files = os.listdir(datasets_path)
    
    for file in files:
        data_csv = datasets_path+"/"+file
        df_current = pd.read_csv(data_csv).fillna("")        
        df_all = pd.concat([df_all,df_current],ignore_index=True)
    
    
    df_all.drop_duplicates(subset=['Code', 'CWE_ID'], inplace=True)
    
    df_all.to_csv(round_path+"/Scanned_"+APP_FILES+"_Unprocessed_Dataset.csv")    
    print (df_all.shape)

In [12]:
combine_dfs()

(118922, 8)
